In [10]:
# 魔术指令，自动加载模块
%load_ext autoreload
%autoreload 2

import os

os.chdir("/home/beihang/xihu/HZTourism")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import pandas as pd

df = pd.read_csv("data/ogn/mobile_signaling_tourists_num.csv")
# 筛选spot_id=14100的数据
df = df[df["spot_id"] == 14100]
df["kpi_time"] = pd.to_datetime(df["kpi_time"])
# 按照kpi_time升序排序
df = df.sort_values("kpi_time")
# 数据按时间去重
df = df.drop_duplicates("kpi_time")

In [8]:
df.head(), df.tail()

(      id  spot_id  kpi_id            kpi_time  kpi_value
 0     63    14100    1001 2021-09-01 00:00:00       9752
 58   121    14100    1001 2021-09-07 15:00:00      32346
 116  179    14100    1001 2021-09-07 15:05:00      32346
 174  237    14100    1001 2021-09-07 15:10:00      30302
 232  295    14100    1001 2021-09-07 15:15:00      30302,
                 id  spot_id  kpi_id            kpi_time  kpi_value
 19930506  39858527    14100    1001 2025-04-11 14:10:00      55238
 19930562  39858639    14100    1001 2025-04-11 14:15:00      55715
 19930618  39858751    14100    1001 2025-04-11 14:20:00      55715
 19930674  39858863    14100    1001 2025-04-11 14:25:00      55207
 19930730  39858975    14100    1001 2025-04-11 14:30:00      55207)

In [3]:
from utils_data.utils_data import (
    filter_days_with_less_consecutive_missing,
    fill_missing_value_singlespot_day,
)

# 分别筛选出2024年的数据以及2025年的数据
df_2024 = df[df["kpi_time"].dt.year == 2024]
df_2025 = df[df["kpi_time"].dt.year == 2025]
# 数据去重
df_2024 = df_2024.drop_duplicates(subset=["kpi_time"])
df_2025 = df_2025.drop_duplicates(subset=["kpi_time"])
# 筛选连续数据缺值不多于5条的数据
df_2024 = filter_days_with_less_consecutive_missing(df=df_2024, freq="5min", k=5)
df_2025 = filter_days_with_less_consecutive_missing(df=df_2025, freq="5min", k=5)
# 填充单值
df_2024 = fill_missing_value_singlespot_day(df_2024)
df_2025 = fill_missing_value_singlespot_day(df_2025)
# 去除id列与kpi_id列
df_2024 = df_2024.drop(columns=["id", "kpi_id"])
df_2025 = df_2025.drop(columns=["id", "kpi_id"])
df_2024.to_csv("data/0411/ogn/24/df_2024.csv", index=False)
df_2025.to_csv("data/0411/ogn/25/df_2025.csv", index=False)

In [4]:
# 数据读入，首先按照时间间隔分组，`前后相差一天的分一次组
# 然后按照模式分组，同一模式的分一次组
df_2024 = pd.read_csv("data/0411/ogn/24/df_2024.csv")
df_2025 = pd.read_csv("data/0411/ogn/25/df_2025.csv")

In [ ]:
import numpy as np
import pandas as pd
import os


# 获取分组的编码标注数据,返回两个dataframe列表
def get_group_annotation(df, his_len=288, pred_len=24):
    # 转换kpi_time为datetime类型
    df["kpi_time"] = pd.to_datetime(df["kpi_time"])
    # 对kpi_time进行升序排序
    df = df.sort_values(by="kpi_time")
    df["mode"] = df["kpi_time"].dt.weekday.apply(lambda x: 1 if x < 5 else 0)
    # 按天进行分组
    daily_groups = df.groupby(df["kpi_time"].dt.date)
    # 遍历每个分组，如果前后两个分组的时间差为一天，则将它们合并，否则新键一个分组
    all_groups = []
    # 先添加第一个分组

    prev_group_name, first_group = next(iter(daily_groups))
    all_groups.append(first_group)

    # 从第二个分组开始遍历
    for i, (group_name, group) in enumerate(daily_groups):
        if i > 0:
            if (group_name - prev_group_name).days == 1:
                all_groups[-1] = pd.concat([all_groups[-1], group])
            else:
                all_groups.append(group)
            prev_group_name = group_name
    # 遍历每个分组，从每个分组的第his_len个元素开始
    groups_mode_0 = []
    groups_mode_1 = []
    for group in all_groups:
        if len(group) < his_len + pred_len:
            continue
        # 第一组样例
        # 获取mode第his_len到his_len+pred_len个元素
        data_temp = group[: his_len + pred_len]
        modes = group["mode"][his_len : his_len + pred_len]
        # 统计一下，看看mode=0与mode=1的个数
        num_mode_0 = np.sum(modes == 0)
        num_mode_1 = pred_len - num_mode_0
        if num_mode_1 >= num_mode_0:
            mode = 1
        else:
            mode = 0
        mode_pre = mode
        # 遍历每个分组，从每个分组的第his_len个元素开始
        for i in range(his_len, len(group) - pred_len + 1):
            # 获取第i个元素
            data_i = group.iloc[i]
            # 获取第i+pred_len个元素
            data_j = group.iloc[i + pred_len - 1]
            if data_i["mode"] != data_j["mode"]:
                if data_i["mode"] == 1:
                    num_mode_1 -= 1
                    num_mode_0 += 1
                else:
                    num_mode_0 -= 1
                    num_mode_1 += 1
            # 获取训练集组合
            if mode == mode_pre:
                data_temp = pd.concat([data_temp, data_j.to_frame().T])
                if i == len(group) - pred_len:
                    if mode_pre == 0:
                        groups_mode_0.append(data_temp)
                    else:
                        groups_mode_1.append(data_temp)
            else:
                if mode_pre == 0:
                    groups_mode_0.append(data_temp)
                else:
                    groups_mode_1.append(data_temp)
                # 前his_len到当前元素
                data_temp = group[i - his_len : i]
            mode_pre = mode
    return groups_mode_0, groups_mode_1


# 保存每种模式的数据
def save_mode_data(groups_mode, mode, his_len, data_basepath="data/0411/ogn/24"):
    data_basepath = f"{data_basepath}/{str(his_len)}/mode_{mode}"
    if not os.path.exists(data_basepath):
        os.makedirs(data_basepath)
    for i, df_temp in enumerate(groups_mode):
        # 按kpi_time去重
        df_temp = df_temp.drop_duplicates(subset=["kpi_time"])
        df_temp.to_csv(f"{data_basepath}/{str(i)}.csv", index=False)

In [6]:
for df, data_basepath in zip(
    [df_2024, df_2025], ["data/0411/ogn/24", "data/0411/ogn/25"]
):
    for his_len in [288, 2016]:
        groups_mode_0, groups_mode_1 = get_group_annotation(his_len=his_len, df=df)
        save_mode_data(
            groups_mode=groups_mode_0,
            mode=0,
            his_len=his_len,
            data_basepath=data_basepath,
        )
        save_mode_data(
            groups_mode=groups_mode_1,
            mode=1,
            his_len=his_len,
            data_basepath=data_basepath,
        )

In [9]:
# 按照数据连续性进行分组
def get_continuous_group(
    df, save_dir="data/0411/ogn/24/mode_no", his_len=288, pred_len=24
):
    # 转换kpi_time为datetime类型
    df["kpi_time"] = pd.to_datetime(df["kpi_time"])
    # 对kpi_time进行升序排序
    df = df.sort_values(by="kpi_time")
    daily_groups = df.groupby(df["kpi_time"].dt.date)
    all_groups = []
    prev_group_name, first_group = next(iter(daily_groups))
    all_groups.append(first_group)

    # 从第二个分组开始遍历
    for i, (group_name, group) in enumerate(daily_groups):
        if i > 0:
            if (group_name - prev_group_name).days == 1:
                all_groups[-1] = pd.concat([all_groups[-1], group])
            else:
                all_groups.append(group)
            prev_group_name = group_name
    for i, group in enumerate(all_groups):
        if len(group) >= his_len + pred_len:
            group.to_csv(f"{save_dir}/{i}.csv", index=False)

In [11]:
df_2024 = pd.read_csv("data/0411/ogn/24/df_2024.csv")
get_continuous_group(df=df_2024, save_dir="data/0411/ogn/24/mode_no")

In [14]:
df_2025 = pd.read_csv("data/0411/ogn/25/df_2025.csv")
get_continuous_group(df=df_2025, save_dir="data/0411/ogn/25/mode_no")